<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Juypter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Numerical methods: 7. Linear systems
## Sparse systems
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

In this notebook, we discuss matrix storage for larger systems of linear equations.
Here, storing a full $ n \times n$ matrix becomes inefficient.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
import scipy.sparse
from sys import getsizeof

----
## Compressed row storage

We follow here the [wikipedia](https://de.wikipedia.org/wiki/Compressed_Row_Storage) entry 
to **compressed row storage**, a method to keep only the **non-zero** entries of a matrix in memory.

Consider the 4x5-matrix:

$$
A=
  \begin{pmatrix} 
    \underset{({\color{Blue}0},{\color{Green}0})}{10} & 0 & 0 & \underset{({\color{Blue}0},{\color{Green}3})}{12} & 0 \\ 
    0 & 0 & \underset{({\color{Blue}1},{\color{Green}2})}{11} & 0 & \underset{({\color{Blue}1},{\color{Green}4})}{13} \\
    0 & \underset{({\color{Blue}2},{\color{Green}1})}{16} & 0 & 0 & 0 \\
    0 & 0 & \underset{({\color{Blue}3},{\color{Green}2})}{11} & 0 & \underset{({\color{Blue}3},{\color{Green}4})}{13} \\
  \end{pmatrix}
$$
In every row, we have only one or two non-zero entries, which we store in a 1D array:
$$
A_{sparse} = \left( 10, 12, 11, 13, 16, 11, 13 \right)
$$
We then need to use to integer arrays to store the locations of the non-zeros entries.
Ther first array, $icol$, markes the column positions of each entry (the green enties above):
$$
icol = \left( 0,3,2,4,1,2,4 \right)
$$
To identify, to which row these colunm indices belong, a second integer array $irow$ is used:
$$
irow = \left( 0,2,4,5,7 \right)
$$
This array marks the starting position of column entries for a specific row in $icol$.

We recover the positions for the non-zero entries with a small `python` script:

In [ ]:
Awiki = np.array([10,12,11,13,16,11,13],dtype=float)
icol = np.array([0,3,2,4,1,2,4],dtype=int)
irow = np.array([0,2,4,5,7],dtype=int)

print('i j A[i,j]')
for i in range(len(irow)-1):
    for j in range(irow[i],irow[i+1]):
        print(i,icol[j],Awiki[j])

----
## Example of full random matrix

We create a $n \times n$ matrix $A$ with random entries and a right-hand side vector $b$,
also filled with random values. Note that we fixed the random seed to replicate the random numbers.

We use the `plt.spy` routine from `matplotlib` to visualise the storage of non-zero
elements of the matrix A.

In [ ]:
n = 10
np.random.seed(11)
A = np.random.random(n*n).reshape(n,n)
b = np.random.random(n)

plt.spy(A,marker='o', markersize=9)

We solve this random system with `solve` from the `scipy.linalg` package (which is similar to the
one in the `numpy` package).

In [ ]:
x = scipy.linalg.solve(A,b)
print('x:\n',x)
print(np.allclose(np.dot(A,x),b))

We check to storage size occupied by matrix $A$ with `getsize` from the `sys` package:

In [ ]:
print(getsizeof(A),' bytes')

----
## Example with banded matrix

Next, we create a **banded matrix** with 
- the main diagonal filled with $-2$,
- both first side diagonals filled with $1$,
- otherwise entries are zero.

We use `np.ones`first to create an 1D array with the non-zero values for each diagonal,
then expand these non-zero entries into a 2D array, using `np.diag`.

In [ ]:
print(np.ones(n))
print(np.diag(np.ones(n)))

In [ ]:
np.random.seed(11)
A = np.diag(-2*np.ones(n)) + np.diag(np.ones(n-1),k=1) + np.diag(np.ones(n-1),k=-1)
b = np.random.random(n)
print(A)
plt.spy(A,marker='o', markersize=9)

print(getsizeof(A),' bytes')

We solve this banded system again with `solve` from the `scipy.linalg` package (which is similar to the
one in the `numpy` package).

In [ ]:
x = scipy.linalg.solve(A,b)
print('x:\n',x)
print(np.allclose(np.dot(A,x),b))

----
## Sparse storage and sparse solver

We rearrange the banded matrix $A$ into a **compressed row storage**, using the
`csr_matrix` routine from the `scipy.sparse` package.

With `print()`, we check to sparse matrix, and with the function `toarray()`, we can convert
it back to full storage (which we need for the check of the solution later ...).

In [ ]:
Asparse = scipy.sparse.csr_matrix(A)
print(getsizeof(Asparse),' bytes')
print('A:\n',Asparse)
print('A:\n',Asparse.toarray())
print('b:\n',b)

plt.spy(Asparse)

In [ ]:
x =scipy.sparse.linalg.spsolve(Asparse,b)
print('x:\n',x)
print(np.allclose(np.dot(Asparse.toarray(),x),b))

----
## Check the wikipedia example ...

In [ ]:
A = np.array([[10,0,0,12,0],[0,0,11,0,13],[0,16,0,0,0],[0,0,11,0,13]])
Asparse = scipy.sparse.csr_matrix(A)
print(Asparse)

----
## Condition number

Test the **condition number** of a matrix. Use as example
$$
A = \left[
\begin{array}{cc}
1 & 2 \\
1.0001 & 2
\end{array}
\right]
$$
The condition number $K$ can be calculated from the matrix norm:
$$
K(A) = ||A|| \cdot ||A^{-1}||
$$
Here, as norm, the $l_2$ norm, the $l_{\infty}$ norm, or another norm can be used.

In [ ]:
A = np.array([[1,2],[1.0001,2]],dtype=float)
K    = np.linalg.cond(A)
K2   = np.linalg.cond(A,2)
Kinf = np.linalg.cond(A,np.inf)

print('K: ',round(K,2), ' default, L2 norm')
print('K: ',round(K2,2),' L2 norm')
print('K: ',round(Kinf,2),' Linfty norm')

----